# 🎙️ Entrenamiento de Voz Piper TTS en Google Colab

Este notebook permite entrenar modelos de voz Piper TTS con GPU gratuita de Google Colab.

**GPU recomendada:** T4 o superior  
**Tiempo por época:** ~10-30 minutos con GPU (vs 30-60 min en CPU)

---

## 1️⃣ Configuración Inicial

Verificar GPU y montar Google Drive

In [ ]:
# Verificar GPU disponible
!nvidia-smi

import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

: 

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Crear directorio de trabajo
!mkdir -p /content/piper-training
%cd /content/piper-training

## 2️⃣ Instalación de Dependencias

Instalar Piper y todas las dependencias

In [ ]:
%%bash
set -e
echo "📦 Instalando dependencias del sistema..."
apt-get update -qq
apt-get install -y -qq espeak-ng wget git 2>&1 | grep -v "debconf"
echo "✅ espeak-ng instalado"
espeak-ng --version

In [ ]:
%%bash
echo "📦 Instalando piper-phonemize..."
cd /content/piper-training

if [ ! -d "piper_phonemize" ]; then
    echo "Descargando piper-phonemize..."
    wget --show-progress \
        https://github.com/rhasspy/piper-phonemize/releases/download/2023.11.14-4/piper-phonemize_linux_x86_64.tar.gz \
        || { echo "❌ Error descargando piper-phonemize"; exit 1; }
    
    echo "Descomprimiendo..."
    tar -xzf piper-phonemize_linux_x86_64.tar.gz \
        || { echo "❌ Error descomprimiendo piper-phonemize"; exit 1; }
    
    rm piper-phonemize_linux_x86_64.tar.gz
    echo "✅ piper-phonemize instalado"
    
    # Verificar instalación
    if [ -d "piper_phonemize" ]; then
        echo "✅ Directorio piper_phonemize creado correctamente"
        ls -la piper_phonemize/ | head -10
    fi
else
    echo "✅ piper-phonemize ya existe"
    ls -la piper_phonemize/ | head -10
fi

In [ ]:
%%bash
set -e
echo "📦 Clonando Piper..."
if [ ! -d "piper" ]; then
    git clone -q https://github.com/rhasspy/piper.git
    echo "✅ Piper clonado"
else
    echo "✅ Piper ya existe"
fi

In [ ]:
# ⚠️ EJECUTAR ESTA CELDA Y LUEGO REINICIAR EL RUNTIME
# Después de completar, ve a: Runtime > Restart session
# Luego ejecuta la siguiente celda de verificación

import os
import sys
from pathlib import Path

print("📦 INSTALACIÓN DE DEPENDENCIAS")
print("=" * 50)
print("⚠️ Al finalizar, REINICIA el runtime:")
print("   Runtime > Restart session")
print("=" * 50 + "\n")

# Bajar versión de pip
print("📦 Ajustando pip...")
!pip install --quiet pip==24.0

# Instalar PyTorch con CUDA 12.1
print("\n📦 Instalando PyTorch 2.2.0...")
!pip install --quiet torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu121

# Instalar dependencias
print("\n📦 Instalando dependencias...")
!pip install --quiet "pytorch-lightning>=1.9.0,<2.0.0"
!pip install --quiet "librosa>=0.9.0"
!pip install --quiet "onnxruntime>=1.10.0"
!pip install --quiet "cython>=0.29.0"
!pip install --quiet "pydantic>=1.10.0,<2.0"
!pip install --quiet "onnx>=1.11.0"

# Configurar piper-phonemize
print("\n📦 Configurando piper-phonemize...")
phonemize_lib = "/content/piper-training/piper_phonemize/lib"
if Path(phonemize_lib).exists():
    print(f"✅ Binarios encontrados")
else:
    for so_file in Path("/content/piper-training").rglob("libpiper_phonemize.so*"):
        phonemize_lib = str(so_file.parent)
        break

# Guardar config en archivo para después del reinicio
config_file = "/content/piper-training/.piper_config"
with open(config_file, 'w') as f:
    f.write(f"PHONEMIZE_LIB={phonemize_lib}\n")
    f.write(f"PIPER_PYTHON=/content/piper-training/piper/src/python\n")
print(f"✅ Configuración guardada")

# Instalar piper_train
print("\n📦 Instalando piper_train...")
piper_python_dir = "/content/piper-training/piper/src/python"
%cd {piper_python_dir}
!pip install --quiet -e . --no-deps
%cd /content/piper-training

# Forzar numpy y scipy correctos AL FINAL
print("\n📦 Instalando numpy 1.26.4 y scipy 1.11.4...")
!pip uninstall -y numpy scipy 2>/dev/null
!pip install --quiet --force-reinstall "numpy==1.26.4"
!pip install --quiet --force-reinstall "scipy==1.11.4"

print("\n" + "=" * 50)
print("✅ INSTALACIÓN COMPLETADA")
print("=" * 50)
print("\n⚠️ IMPORTANTE: Ahora REINICIA el runtime:")
print("   Ve a: Runtime > Restart session")
print("   Luego ejecuta la SIGUIENTE CELDA para verificar")
print("=" * 50)

In [ ]:
# ✅ EJECUTAR DESPUÉS DE REINICIAR EL RUNTIME
# Esta celda configura el entorno y verifica la instalación

import os
import sys
from pathlib import Path

print("🔄 Configurando entorno después del reinicio...\n")

# Leer configuración guardada
config_file = "/content/piper-training/.piper_config"
config = {}
if Path(config_file).exists():
    with open(config_file, 'r') as f:
        for line in f:
            key, value = line.strip().split('=', 1)
            config[key] = value

# Configurar variables de entorno
phonemize_lib = config.get('PHONEMIZE_LIB', '/content/piper-training/piper_phonemize/lib')
piper_python = config.get('PIPER_PYTHON', '/content/piper-training/piper/src/python')

os.environ['LD_LIBRARY_PATH'] = f"{phonemize_lib}:{os.environ.get('LD_LIBRARY_PATH', '')}"
os.environ['PATH'] = f"{phonemize_lib}:{os.environ.get('PATH', '')}"
os.environ['PYTHONPATH'] = f"{piper_python}:{os.environ.get('PYTHONPATH', '')}"
sys.path.insert(0, piper_python)

# Cambiar al directorio de trabajo
%cd /content/piper-training

# Verificación
print("🔍 VERIFICACIÓN DEL ENTORNO")
print("=" * 50)

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

import numpy as np
import scipy
print(f"\nnumpy: {np.__version__}")
print(f"scipy: {scipy.__version__}")

# Verificar piper_train
print("\n🔍 Módulos piper_train:")
try:
    import piper_train
    print("✅ piper_train")
    from piper_train import vits
    print("✅ piper_train.vits")
except Exception as e:
    print(f"❌ Error: {e}")

# Verificación final
if np.__version__.startswith("1.26") and scipy.__version__.startswith("1.11"):
    print("\n" + "=" * 50)
    print("✅ ENTORNO LISTO PARA ENTRENAR")
    print("=" * 50)
else:
    print("\n⚠️ Las versiones no son correctas.")
    print("   Ejecuta la celda anterior y reinicia el runtime.")

## 3️⃣ Descargar Modelo Base

In [ ]:
%%bash
set -e
mkdir -p models_base && cd models_base
if [ ! -f "en_US-lessac-high.ckpt" ]; then
    echo "📥 Descargando checkpoint (952 MB)..."
    wget -q --show-progress \
        "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/7bf647cb000d8c8319c6cdd4289dd6b7d0d3eeb8/en/en_US/lessac/high/epoch=2218-step=838782.ckpt" \
        -O en_US-lessac-high.ckpt
    echo "✅ Checkpoint descargado"
else
    echo "✅ Checkpoint ya existe"
fi

## 4️⃣ Configurar Dataset

Sube tu dataset preprocesado o procesa uno nuevo

In [ ]:
# OPCIÓN A: Copiar desde Google Drive
DRIVE_PATH = "/content/drive/MyDrive/sig"  # ⬅️ AJUSTAR

!mkdir -p datasets
!cp -r "{DRIVE_PATH}" datasets/
import os
DATASET_NAME = os.path.basename(DRIVE_PATH)
DATASET_DIR = f"datasets/{DATASET_NAME}"
print(f"✅ Dataset copiado: {DATASET_DIR}")

In [ ]:
# OPCIÓN B: Subir ZIP y preprocesar
from google.colab import files
uploaded = files.upload()  # Sube tu dataset.zip aquí

In [ ]:
# OPCIÓN C: Preprocesar dataset directamente en Colab
# Si subes un ZIP con formato LJSpeech (wavs/ y metadata.csv)
import zipfile
import os

# Descomprimir dataset
zip_file = "dataset.zip"  # ⬅️ Nombre del archivo subido
if os.path.exists(zip_file):
    print(f"📦 Descomprimiendo {zip_file}...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall("datasets/raw")
    
    # Preprocesar dataset
    RAW_DIR = "datasets/raw/mi_dataset"  # ⬅️ AJUSTAR nombre
    PROCESSED_DIR = "datasets/sig"
    LANGUAGE = "es-es"  # ⬅️ AJUSTAR idioma
    
    print(f"⚙️ Preprocesando dataset...")
    !python -m piper_train.preprocess \
      --language {LANGUAGE} \
      --input-dir {RAW_DIR} \
      --output-dir {PROCESSED_DIR} \
      --single-speaker \
      --sample-rate 22050
    
    DATASET_DIR = PROCESSED_DIR
    print(f"✅ Dataset preprocesado: {DATASET_DIR}")
else:
    print("⚠️ No se encontró dataset.zip, asegúrate de subirlo primero")

### 📋 Verificar Dataset

Antes de entrenar, verifica la estructura del dataset

In [ ]:
# Verificar estructura del dataset
import json
from pathlib import Path

DATASET_DIR = "datasets/sig"  # ⬅️ AJUSTAR si es necesario
dataset_path = Path(DATASET_DIR)

print("🔍 Verificando dataset...\n")

# Verificar archivos requeridos
required_files = ['config.json', 'dataset.jsonl']
for file in required_files:
    file_path = dataset_path / file
    if file_path.exists():
        print(f"✅ {file}")
        if file == 'config.json':
            with open(file_path, 'r') as f:
                config = json.load(f)
                print(f"   - Audio: {config.get('audio', {}).get('sample_rate')} Hz")
                print(f"   - Idioma: {config.get('espeak', {}).get('voice')}")
    else:
        print(f"❌ {file} no encontrado")

# Contar muestras
jsonl_path = dataset_path / 'dataset.jsonl'
if jsonl_path.exists():
    with open(jsonl_path, 'r') as f:
        num_samples = sum(1 for _ in f)
    print(f"\n📊 Total de muestras: {num_samples}")
    
    # Calcular duración aproximada de entrenamiento
    samples_per_epoch = num_samples
    time_per_sample = 0.3  # segundos aproximados por muestra en GPU T4
    time_per_epoch = (samples_per_epoch * time_per_sample) / 60
    print(f"⏱️ Tiempo estimado por época: ~{time_per_epoch:.1f} minutos (GPU T4)")
else:
    print("❌ No se pudo contar muestras")

## 5️⃣ Entrenar Modelo

In [ ]:
# Configuración de entrenamiento
DATASET_DIR = "datasets/sig"  # ⬅️ AJUSTAR
MAX_EPOCHS = 100
BATCH_SIZE = 16  # Ajustar según VRAM disponible
CHECKPOINT_EPOCHS = 5
VALIDATION_SPLIT = 0.05
NUM_TEST_EXAMPLES = 0
PRECISION = "32"  # Usar "32" para mejor compatibilidad

# Entrenar modelo con piper_train
!python -m piper_train \
  --dataset-dir {DATASET_DIR} \
  --accelerator gpu \
  --devices 1 \
  --batch-size {BATCH_SIZE} \
  --validation-split {VALIDATION_SPLIT} \
  --num-test-examples {NUM_TEST_EXAMPLES} \
  --max_epochs {MAX_EPOCHS} \
  --resume_from_checkpoint models_base/en_US-lessac-high.ckpt \
  --checkpoint-epochs {CHECKPOINT_EPOCHS} \
  --precision {PRECISION}

### ⚙️ Configuración Avanzada

Ajusta estos parámetros según tu caso:

- **BATCH_SIZE**: 
  - 16-32 para T4 (16GB)
  - 8-16 para GPUs con menos VRAM
  - Reduce si obtienes errores de memoria
  
- **MAX_EPOCHS**: 
  - Mínimo 100 para resultados aceptables
  - 500-1000+ para mejor calidad
  
- **CHECKPOINT_EPOCHS**: Cada cuántas épocas guardar un checkpoint

## 6️⃣ Monitorear Progreso

In [ ]:
# Monitorear el progreso durante el entrenamiento
import time
from IPython.display import clear_output

def monitor_training(dataset_dir, update_interval=30):
    """
    Monitorea el entrenamiento en tiempo real
    
    Args:
        dataset_dir: Directorio del dataset
        update_interval: Segundos entre actualizaciones
    """
    import pandas as pd
    from pathlib import Path
    
    logs_dir = Path(dataset_dir) / "lightning_logs"
    
    while True:
        try:
            clear_output(wait=True)
            
            # Buscar última versión
            versions = sorted(logs_dir.glob("version_*"))
            if not versions:
                print("⏳ Esperando inicio del entrenamiento...")
                time.sleep(update_interval)
                continue
            
            metrics_file = versions[-1] / "metrics.csv"
            if metrics_file.exists():
                df = pd.read_csv(metrics_file)
                
                print("=" * 60)
                print("📊 PROGRESO DEL ENTRENAMIENTO")
                print("=" * 60)
                print(f"\nÉpoca actual: {int(df['epoch'].max())}")
                print(f"Total de pasos: {int(df['step'].max())}")
                print(f"\n--- Últimas 5 métricas ---")
                print(df[['epoch', 'loss_gen_all', 'loss_disc_all']].tail())
                print("\n💡 Presiona el botón STOP para detener el monitoreo")
                
            time.sleep(update_interval)
            
        except KeyboardInterrupt:
            print("\n✅ Monitoreo detenido")
            break
        except Exception as e:
            print(f"⚠️ Error: {e}")
            time.sleep(update_interval)

# Para iniciar el monitoreo (ejecutar en otra celda mientras entrena):
# monitor_training(DATASET_DIR, update_interval=30)

In [ ]:
# Ver métricas
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

logs_dir = Path(DATASET_DIR) / "lightning_logs"
versions = sorted(logs_dir.glob("version_*"))
if versions:
    metrics = versions[-1] / "metrics.csv"
    if metrics.exists():
        df = pd.read_csv(metrics)
        print(df.tail())
        
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(df['epoch'], df['loss_gen_all'])
        plt.title('Generator Loss')
        plt.subplot(1, 2, 2)
        plt.plot(df['epoch'], df['loss_disc_all'])
        plt.title('Discriminator Loss')
        plt.tight_layout()
        plt.show()

## 7️⃣ Exportar Modelo

In [ ]:
# Exportar a ONNX
from pathlib import Path

# Buscar el último checkpoint
logs_dir = Path(DATASET_DIR) / "lightning_logs"
versions = sorted(logs_dir.glob("version_*"))

if not versions:
    print("❌ No se encontraron versiones de entrenamiento")
else:
    latest_version = versions[-1]
    checkpoints = sorted(latest_version.glob("checkpoints/*.ckpt"))
    
    if not checkpoints:
        print("❌ No se encontraron checkpoints")
    else:
        CHECKPOINT = str(checkpoints[-1])
        print(f"📦 Exportando checkpoint: {CHECKPOINT}")
        
        # Crear directorio de salida
        !mkdir -p outputs
        
        # Exportar a ONNX usando el módulo correcto
        !python3 -m piper_train.export_onnx \
            "{CHECKPOINT}" \
            "/content/piper-training/outputs/model.onnx"
        
        # Copiar archivo de configuración JSON
        config_src = Path(DATASET_DIR) / "config.json"
        if config_src.exists():
            !cp "{config_src}" outputs/model.onnx.json
            print("✅ Modelo exportado a outputs/")
            print(f"   - model.onnx")
            print(f"   - model.onnx.json")
        else:
            print("⚠️ No se encontró config.json en el dataset")

## 8️⃣ Descargar Resultados

In [ ]:
# Guardar en Google Drive
!mkdir -p "/content/drive/MyDrive/piper-models/trained_model"
!cp -r outputs/* "/content/drive/MyDrive/piper-models/trained_model/"
print("✅ Guardado en Drive")

In [ ]:
# O descargar directamente
from google.colab import files
!zip -r model_trained.zip outputs/
files.download("model_trained.zip")

## 9️⃣ Probar el Modelo

Prueba tu modelo entrenado directamente en Colab

In [ ]:
# Instalar piper-tts para pruebas
!pip install -q piper-tts

# Generar audio de prueba
TEST_TEXT = "Hola, esta es una prueba de mi voz personalizada con Piper TTS."
OUTPUT_WAV = "prueba.wav"

!echo "{TEST_TEXT}" | piper --model outputs/model.onnx --output_file {OUTPUT_WAV}

# Reproducir audio
from IPython.display import Audio, display
import os

if os.path.exists(OUTPUT_WAV):
    print("✅ Audio generado exitosamente")
    display(Audio(OUTPUT_WAV, autoplay=False))
else:
    print("❌ No se pudo generar el audio")

## 🔟 Solución de Problemas

### Errores comunes y soluciones:

**1. Error de memoria GPU (CUDA out of memory)**
- Reduce `BATCH_SIZE` (prueba con 8, 4, o incluso 2)
- Cierra otras celdas que usen GPU
- Reinicia el runtime: `Runtime > Restart runtime`

**2. Error "No module named 'piper_train'"**
- Verifica que ejecutaste la celda de instalación de dependencias
- Asegúrate de estar en el directorio correcto

**3. Dataset vacío o sin muestras**
- Verifica que `dataset.jsonl` tenga contenido
- Revisa que los archivos de audio estén en el formato correcto
- Ejecuta la celda de verificación de dataset

**4. Pérdidas (losses) no disminuyen**
- Es normal al principio del entrenamiento
- Espera al menos 50-100 épocas antes de evaluar
- Verifica que el modelo base sea compatible con tu idioma

**5. Entrenamiento muy lento**
- Verifica que estés usando GPU: `Runtime > Change runtime type > GPU`
- Comprueba disponibilidad con `!nvidia-smi`
- GPU T4 es recomendada para mejor rendimiento

## 🧹 Limpieza (Opcional)

Libera espacio eliminando archivos temporales

In [ ]:
# Eliminar archivos temporales para liberar espacio
import shutil
from pathlib import Path

print("🧹 Limpiando archivos temporales...\n")

# Eliminar logs antiguos (mantener solo el último)
logs_dir = Path(DATASET_DIR) / "lightning_logs"
if logs_dir.exists():
    versions = sorted(logs_dir.glob("version_*"))
    if len(versions) > 1:
        for old_version in versions[:-1]:
            shutil.rmtree(old_version)
            print(f"✅ Eliminado: {old_version.name}")

# Eliminar checkpoints intermedios (mantener solo cada 10 épocas)
if versions:
    checkpoints_dir = versions[-1] / "checkpoints"
    if checkpoints_dir.exists():
        checkpoints = sorted(checkpoints_dir.glob("*.ckpt"))
        for ckpt in checkpoints[:-1]:  # Mantener el último
            epoch = ckpt.stem.split('-')[0].replace('epoch=', '')
            if epoch.isdigit() and int(epoch) % 10 != 0:
                ckpt.unlink()
                print(f"✅ Eliminado checkpoint: {ckpt.name}")

# Verificar espacio disponible
total, used, free = shutil.disk_usage("/")
print(f"\n💾 Espacio disponible: {free // (2**30)} GB")
print("✅ Limpieza completada")